In [1]:
include("LiPoSID.jl")
using LinearAlgebra

Consider Kraus operators for the Amplitude damping

$\begin{bmatrix}
1 & 0 \\ 0 & \sqrt{1-\gamma}
\end{bmatrix}, \quad \begin{bmatrix}
0 & \sqrt{\gamma} \\ 0 & 0 \end{bmatrix}$

In [2]:
γ = 0.4
E = []
push!(E, [1 0
          0 sqrt(1-γ)])
        
push!(E, [0 sqrt(γ)
          0  0 ])

sum(e'e for e in E)

ρᵍ₀ = [1. 0]' * [1 0]
ρᵉ₀ = [0. 1]' * [0 1]
ρˣ₀ = [1 1]' * [1 1]/2
ρʸ₀ = [1 -im]' * [1 -im]/2

ρᵍ₁ = LiPoSID.Kraus(ρᵍ₀, E); @assert ishermitian(ρᵍ₁)
ρᵉ₁ = LiPoSID.Kraus(ρᵉ₀, E); @assert ishermitian(ρᵉ₁)
ρˣ₁ = LiPoSID.Kraus(ρˣ₀, E); @assert ishermitian(ρˣ₁)
ρʸ₁ = LiPoSID.Kraus(ρʸ₀, E); @assert ishermitian(ρʸ₁)

In [3]:
E

2-element Vector{Any}:
 [1.0 0.0; 0.0 0.7745966692414834]
 [0.0 0.6324555320336759; 0.0 0.0]

In [4]:
Eᵗᵒᵐ = LiPoSID.QPT(ρᵍ₁, ρᵉ₁, ρˣ₁, ρʸ₁)

4-element Vector{Any}:
 ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]
 ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]
 ComplexF64[0.0 + 0.0im 0.6324555320336758 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]
 ComplexF64[-1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.7745966692414833 + 0.0im]

In [5]:
sum(e'e for e in Eᵗᵒᵐ)

2×2 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

In [6]:
t_steps = 20

ρᵍ = LiPoSID.timeevolution_kraus(t_steps, ρᵍ₀, E)
ρᵉ = LiPoSID.timeevolution_kraus(t_steps, ρᵉ₀, E)
ρˣ = LiPoSID.timeevolution_kraus(t_steps, ρˣ₀, E)
ρʸ = LiPoSID.timeevolution_kraus(t_steps, ρʸ₀, E)
A = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ, ρᵍ, ρˣ, ρʸ])

4×4 Matrix{Float64}:
  0.774597     8.11238e-16   3.03968e-16  -1.21929e-16
  1.04284e-16  0.774597     -1.24035e-16   5.68155e-17
 -8.64112e-16  1.12484e-15   0.6           0.4
 -4.23273e-16  2.16493e-15   3.36536e-16   1.0

In [7]:
function bloch4(ρ)

    b = convert.(Float64, [ ρ[1,2] + ρ[2,1],
                           (ρ[1,2] - ρ[2,1])*im,    #ρ[2,1] - ρ[1,2] ?
                            ρ[1,1] - ρ[2,2],
                               1                 ]) #ρ[1,1] + ρ[2,2]  

end

function dm_b4(b) 

    ρ = [ 1+b[3]         b[1]-im*b[2]
          b[1]+im*b[2]   1-b[3]       ]/2

end

@assert dm_b4(bloch4(ρᵍ₀)) ≈ ρᵍ₀
@assert dm_b4(bloch4(ρᵉ₀)) ≈ ρᵉ₀
@assert dm_b4(bloch4(ρˣ₀)) ≈ ρˣ₀
@assert dm_b4(bloch4(ρʸ₀)) ≈ ρʸ₀

In [9]:
@assert dm_b4( A  * bloch4(ρᵍ₀)) ≈ ρᵍ₁
@assert dm_b4( A  * bloch4(ρᵉ₀)) ≈ ρᵉ₁
@assert dm_b4( A  * bloch4(ρˣ₀)) ≈ ρˣ₁
@assert dm_b4( A  * bloch4(ρʸ₀)) ≈ ρʸ₁
